# DSE Dissertation

## Topic Analysis

#### Load Packages

In [74]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import datetime as dt
import seaborn as sn

#### Load Dataset

In [75]:
"""

Loading dataset

"""

class UtteranceExample:
    def __init__(self, text, label, do_lower_case):
        self.original_text = text
        self.text = text
        self.label = label

        if do_lower_case:
            self.text = self.text.lower()

    def to_dict(self):
      return {
          'utterance': self.original_text,
          'intent': self.label,
      }
        
def load_utterance_examples(file_path, do_lower_case=True):
    examples = []

    with open('{}/seq.in'.format(file_path), 'r', encoding="utf-8") as f_text, open('{}/label'.format(file_path), 'r', encoding="utf-8") as f_label:
        for text, label in zip(f_text, f_label):
            e = UtteranceExample(text.strip(), label.strip(), do_lower_case)
            examples.append(e)

    return examples



In [76]:
data = load_utterance_examples("/content/sample_data")

utterances= pd.DataFrame.from_records([d.to_dict() for d in data])

print(utterances)

                                              utterance           intent
0                        i am still waiting on my card?     card_arrival
1     what can i do if my card still hasn't arrived ...     card_arrival
2     i have been waiting over a week. is the card s...     card_arrival
3     can i track my card while it is in the process...     card_arrival
4     how do i know if i will get my card, or if it ...     card_arrival
...                                                 ...              ...
8617             you provide support in what countries?  country_support
8618                 what countries are you supporting?  country_support
8619                what countries are getting support?  country_support
8620                     are cards available in the eu?  country_support
8621                   which countries are represented?  country_support

[8622 rows x 2 columns]


### Exploratory Data Analysis

In [77]:
utterances.shape

(8622, 2)

In [78]:
utterances.dropna(axis=0,inplace=True)#dropping na
utterances.shape

(8622, 2)

In [79]:
utterances.drop_duplicates(subset=['utterance'],inplace=True)#dropping duplicates
utterances.shape

(8618, 2)

In [80]:
type_counts_train = utterances['intent'].value_counts()

print(type_counts_train)

"""

THIS SHOWS THE SENTENCES PER INTENT ARE IMBALANCED.

TOTAL OF 77 INTENT CLASSES

"""

card_payment_fee_charged                            167
direct_debit_payment_not_recognised                 162
balance_not_updated_after_cheque_or_cash_deposit    162
wrong_amount_of_cash_received                       160
transaction_charged_twice                           158
                                                   ... 
lost_or_stolen_card                                  66
card_acceptance                                      47
card_swallowed                                       45
virtual_card_not_working                             32
contactless_not_working                              30
Name: intent, Length: 77, dtype: int64


'\n\nTHIS SHOWS THE SENTENCES PER INTENT ARE IMBALANCED.\n\nTOTAL OF 77 INTENT CLASSES\n\n'

In [81]:
utterances.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8618 entries, 0 to 8621
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   utterance  8618 non-null   object
 1   intent     8618 non-null   object
dtypes: object(2)
memory usage: 202.0+ KB


In [82]:
utterances.describe()

,utterance,intent
count,8618,8618
unique,8618,77
top,i am still waiting on my card?,card_payment_fee_charged
freq,1,167


In [83]:
"""

Few samples of the dataset.

"""

utterances.head()

,utterance,intent
0,i am still waiting on my card?,card_arrival
1,what can i do if my card still hasn't arrived ...,card_arrival
2,i have been waiting over a week. is the card s...,card_arrival
3,can i track my card while it is in the process...,card_arrival
4,"how do i know if i will get my card, or if it ...",card_arrival


In [84]:
utterances.groupby('intent').describe()

utterance         \
                                            count unique   
intent                                                     
Refund_not_showing_up                         143    143   
activate_my_card                              141    141   
age_limit                                      92     92   
apple_pay_or_google_pay                       108    108   
atm_support                                    68     68   
...                                           ...    ...   
virtual_card_not_working                       32     32   
visa_or_mastercard                            115    115   
why_verify_identity                           102    102   
wrong_amount_of_cash_received                 160    160   
wrong_exchange_rate_for_cash_withdrawal       145    145   

                                                                                            \
                                                                                       top   
intent                                                                                       
Refund_not_showing_up                    i don't see my refund money yet in my account....   
activate_my_card                          please help me with my card.  it won't activate.   
age_limit                                how old do you need to be to use the banks ser...   
apple_pay_or_google_pay                  hi, i have an apple watch. how do i use it to ...   
atm_support                                             i can use this card at which atms?   
...                                                                                    ...   
virtual_card_not_working                     why isn't my disposable virtual card working?   
visa_or_mastercard                                 is there a way i can have a mastercard?   
why_verify_identity                                     why do you have an identity check?   
wrong_amount_of_cash_received            why did i only receive a partial amount of wha...   
wrong_exchange_rate_for_cash_withdrawal  the wrong exchange rate was applied to me whil...   

                                              
                                        freq  
intent                                        
Refund_not_showing_up                      1  
activate_my_card                           1  
age_limit                                  1  
apple_pay_or_google_pay                    1  
atm_support                                1  
...                                      ...  
virtual_card_not_working                   1  
visa_or_mastercard                         1  
why_verify_identity                        1  
wrong_amount_of_cash_received              1  
wrong_exchange_rate_for_cash_withdrawal    1  

[77 rows x 4 columns]

In [85]:
#Look for missing intents
utterances['intent'].isnull().sum()

0

### Data Pre-processing.
- Spelling correction.
- Remove Punctuation.
- Remove stop words.
- Tokenization.
- Lemmatization.


In [86]:
! pip install -U symspellpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [87]:
"""

Spelling correction using Symspell.


"""
from symspellpy import SymSpell
import pkg_resources
from itertools import islice

sym_spell = SymSpell(max_dictionary_edit_distance=2, prefix_length=7)

dictionary_path = pkg_resources.resource_filename("symspellpy", "frequency_dictionary_en_82_765.txt")
bigram_path = pkg_resources.resource_filename("symspellpy", "frequency_bigramdictionary_en_243_342.txt")

# term_index is the column of the term and count_index is the
# column of the term frequency
sym_spell.load_dictionary(dictionary_path, term_index=0, count_index=1)
sym_spell.load_bigram_dictionary(bigram_path, term_index=0, count_index=2)

# Print out first 5 elements to demonstrate that dictionary is
# successfully loaded
print(list(islice(sym_spell.bigrams.items(), 5)))
print(list(islice(sym_spell.words.items(), 5)))

# lookup suggestions for multi-word input strings (supports compound
# splitting & merging)
input_term = (
    "how do i loacte my card?"
)
# max edit distance per lookup (per single word, not per whole input string)
suggestions = sym_spell.lookup_compound(input_term, max_edit_distance=2)
# display suggestion term, edit distance, and term frequency
for suggestion in suggestions:
    print(suggestion)

def correction(input_term, max_edit_distance=2):
  suggestions = sym_spell.lookup_compound(input_term, max_edit_distance=2)
  #for suggestion in suggestions:
  #  print(suggestion.term)
  return suggestions[0].term

[('abcs of', 10956800), ('aaron and', 10721728), ('abbott and', 7861376), ('abbreviations and', 13518272), ('aberdeen and', 7347776)]
[('the', 23135851162), ('of', 13151942776), ('and', 12997637966), ('to', 12136980858), ('a', 9081174698)]
how do i locate my card, 2, 0


In [88]:
# call the function

corrected_utterances = []

for t in utterances['utterance']:
    corrected_utterances.append(correction(t)) 

corrected_utterances[:5]
utterances['utterance'] = corrected_utterances
utterances.head()

,utterance,intent
0,i am still waiting on my card,card_arrival
1,what can i do if my card still hasn't arrived ...,card_arrival
2,i have been waiting over a week is the card st...,card_arrival
3,can i track my card while it is in the process...,card_arrival
4,how do i know if i will get my card or if it i...,card_arrival


In [89]:
from nltk.corpus import stopwords
import re

import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [90]:
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not",
                           "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not",
                           "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",
                           "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would",
                           "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",
                           "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam",
                           "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have",
                           "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock",
                           "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have",
                           "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",
                           "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",
                           "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",
                           "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",
                           "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",
                           "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are",
                           "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",
                           "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",
                           "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",
                           "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",
                           "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",
                           "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
                           "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",
                           "you're": "you are", "you've": "you have"}

In [91]:
stop_words = set(stopwords.words('english')) 

def text_cleaner(text):
    newString = text.lower()
    newString = re.sub(r'\([^)]*\)', '', newString)   # removing special characters
    newString = re.sub('"','', newString)              # removing '"' 
    newString = ' '.join([contraction_mapping[t] if t in contraction_mapping else t for t in newString.split(" ")])    # replacing slangs
    newString = re.sub(r"'s\b","",newString)    # replacing 's with "" '
    newString = re.sub("[^a-z]", " ", newString)    #only keeping letters
    newString = re.sub('[m]{2,}', 'mm', newString)    # "mmmmmm" -> "mm"
        
    tokens = [w for w in newString.split() if not w in stop_words]     # removing the stopwords

    long_words=[]
    for i in tokens:
        if len(i)>1:                               #removing short word
            long_words.append(i)   
    return (" ".join(long_words)).strip()

In [92]:
#call the function
cleaned_text = []
for t in utterances['utterance']:
    cleaned_text.append(text_cleaner(t)) 

cleaned_text[:5]  

['still waiting card',
 'card still arrived weeks',
 'waiting week card still coming',
 'track card process delivery',
 'know get card lost']

In [93]:
utterances['cleaned_utterance'] = cleaned_text
utterances.head()

,utterance,intent,cleaned_utterance
0,i am still waiting on my card,card_arrival,still waiting card
1,what can i do if my card still hasn't arrived ...,card_arrival,card still arrived weeks
2,i have been waiting over a week is the card st...,card_arrival,waiting week card still coming
3,can i track my card while it is in the process...,card_arrival,track card process delivery
4,how do i know if i will get my card or if it i...,card_arrival,know get card lost


In [94]:
#Drop empty rows

utterances.replace('', np.nan, inplace=True)
utterances.dropna(axis=0,inplace=True)

utterances.head()

,utterance,intent,cleaned_utterance
0,i am still waiting on my card,card_arrival,still waiting card
1,what can i do if my card still hasn't arrived ...,card_arrival,card still arrived weeks
2,i have been waiting over a week is the card st...,card_arrival,waiting week card still coming
3,can i track my card while it is in the process...,card_arrival,track card process delivery
4,how do i know if i will get my card or if it i...,card_arrival,know get card lost


In [95]:
#Lemmatization

nltk.download("wordnet")
nltk.download('omw-1.4')
wordnetLemmatizer = nltk.WordNetLemmatizer()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [96]:
def lemma(tokenized_text):
  text = [wordnetLemmatizer.lemmatize(word) for word in tokenized_text]
  return (" ".join(text)).strip()

utterances['lemmatized_utterance'] = utterances['cleaned_utterance'].apply(lambda x: lemma(x.split(" ")))
utterances.head()

,utterance,intent,cleaned_utterance,lemmatized_utterance
0,i am still waiting on my card,card_arrival,still waiting card,still waiting card
1,what can i do if my card still hasn't arrived ...,card_arrival,card still arrived weeks,card still arrived week
2,i have been waiting over a week is the card st...,card_arrival,waiting week card still coming,waiting week card still coming
3,can i track my card while it is in the process...,card_arrival,track card process delivery,track card process delivery
4,how do i know if i will get my card or if it i...,card_arrival,know get card lost,know get card lost


### Model Building and testing using MultiNomialNB using Count Vectorizer and tf-idf Vectorizer


In [97]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score, f1_score
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

Model building using MultiNomialNB (Naive Bayes) Model and uni-gram CountVectorizer for Classification

In [98]:
"""
Now convert each message into a vector that machine learning models can understand.

"""
# uni-gram CountVectorizer

vectorizer = CountVectorizer().fit_transform(utterances['lemmatized_utterance'])
X = np.array(vectorizer.todense())
print("Shape of X: ", X.shape)
y = utterances['intent']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)
print("Shape of X_train: ", X_train.shape)
print("Shape of X_test: ", X_test.shape)

Shape of X:  (8618, 1811)
Shape of X_train:  (6894, 1811)
Shape of X_test:  (1724, 1811)


In [99]:
count_vectorizer_unigram_MNB_model = MultinomialNB().fit(X_train, y_train)
y_pred = count_vectorizer_unigram_MNB_model.predict(X_test)

y_pred

array(['verify_my_identity', 'beneficiary_not_allowed',
       'transfer_fee_charged', ..., 'declined_cash_withdrawal',
       'balance_not_updated_after_bank_transfer', 'request_refund'],
      dtype='<U48')

In [100]:
print('Accuracy:', accuracy_score(y_test, y_pred))
print('F1 score:', f1_score(y_test, y_pred, average="macro"))

print (classification_report(y_test, y_pred))

Accuracy: 0.7946635730858469
F1 score: 0.7681684912315829
                                                  precision    recall  f1-score   support

                           Refund_not_showing_up       0.81      0.90      0.85        29
                                activate_my_card       0.69      0.86      0.76        28
                                       age_limit       1.00      0.83      0.91        18
                         apple_pay_or_google_pay       0.92      1.00      0.96        22
                                     atm_support       1.00      0.21      0.35        14
                                automatic_top_up       0.95      0.86      0.90        22
         balance_not_updated_after_bank_transfer       0.62      0.80      0.70        30
balance_not_updated_after_cheque_or_cash_deposit       0.78      0.94      0.85        33
                         beneficiary_not_allowed       0.88      0.78      0.82        27
                                 cancel_t

Model building using MultiNomialNB (Naive Bayes) Model and bi-gram CountVectorizer for Classification

In [101]:
# bi-gram CountVectorizer using pipeline approach

X = utterances['lemmatized_utterance']
y = utterances['intent']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

print("Shape of X_train: ", X_train.shape)
print("Shape of X_test: ", X_test.shape)

# Creating a pipeline object
count_vector_MNB_pipeline = Pipeline([
    ('vectorizer_bigrams', CountVectorizer(ngram_range = (2, 2))),        #using the ngram_range parameter 
     ('Multi NB', MultinomialNB())         
])
 
 #fit with features
count_vector_MNB_pipeline.fit(X_train, y_train)


Shape of X_train:  (6894,)
Shape of X_test:  (1724,)


Pipeline(steps=[('vectorizer_bigrams', CountVectorizer(ngram_range=(2, 2))),
                ('Multi NB', MultinomialNB())])

In [102]:
#get the predictions for X_test using pipeline object

y_pred = count_vector_MNB_pipeline.predict(X_test)
y_pred

array(['pending_top_up', 'request_refund', 'transfer_fee_charged', ...,
       'passcode_forgotten', 'cancel_transfer',
       'wrong_exchange_rate_for_cash_withdrawal'], dtype='<U48')

In [103]:
print('Accuracy:', accuracy_score(y_test, y_pred))
print('F1 score:', f1_score(y_test, y_pred, average="macro"))

print (classification_report(y_test, y_pred))

Accuracy: 0.7169373549883991
F1 score: 0.7119539241646126
                                                  precision    recall  f1-score   support

                           Refund_not_showing_up       0.91      0.72      0.81        29
                                activate_my_card       0.76      0.89      0.82        28
                                       age_limit       1.00      0.72      0.84        18
                         apple_pay_or_google_pay       0.92      1.00      0.96        22
                                     atm_support       1.00      0.21      0.35        14
                                automatic_top_up       1.00      0.82      0.90        22
         balance_not_updated_after_bank_transfer       0.62      0.80      0.70        30
balance_not_updated_after_cheque_or_cash_deposit       0.70      0.79      0.74        33
                         beneficiary_not_allowed       0.83      0.56      0.67        27
                                 cancel_t

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Model building using MultiNomialNB (Naive Bayes) Model and uni-gram and bi-gram CountVectorizer for Classification

In [104]:
# uni-gram, bi-gram CountVectorizer

unibigram_vectorizer = CountVectorizer(ngram_range=(1,2)).fit_transform(utterances['lemmatized_utterance'])
X = np.array(unibigram_vectorizer.todense())
print("Shape of X: ", X.shape)
y = utterances['intent']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)
print("Shape of X_train: ", X_train.shape)
print("Shape of X_test: ", X_test.shape)


Shape of X:  (8618, 16561)
Shape of X_train:  (6894, 16561)
Shape of X_test:  (1724, 16561)


In [105]:
ctvectorizer_unibigram_MNB_model = MultinomialNB().fit(X_train, y_train)
y_pred = ctvectorizer_unibigram_MNB_model.predict(X_test)

y_pred

array(['cancel_transfer', 'exchange_via_app',
       'balance_not_updated_after_cheque_or_cash_deposit', ...,
       'get_physical_card', 'reverted_card_payment?',
       'transaction_charged_twice'], dtype='<U48')

In [106]:
print('Accuracy:', accuracy_score(y_test, y_pred))
print('F1 score:', f1_score(y_test, y_pred, average="macro"))

print (classification_report(y_test, y_pred))

Accuracy: 0.8114849187935035
F1 score: 0.8009682298413713
                                                  precision    recall  f1-score   support

                           Refund_not_showing_up       0.96      0.93      0.95        29
                                activate_my_card       0.84      0.96      0.90        28
                                       age_limit       1.00      0.89      0.94        18
                         apple_pay_or_google_pay       0.81      0.95      0.88        22
                                     atm_support       1.00      0.71      0.83        14
                                automatic_top_up       0.95      0.95      0.95        22
         balance_not_updated_after_bank_transfer       0.64      0.77      0.70        30
balance_not_updated_after_cheque_or_cash_deposit       0.74      0.94      0.83        33
                         beneficiary_not_allowed       1.00      0.67      0.80        27
                                 cancel_t

Model building using MultiNomialNB (Naive Bayes) Model and tri-gram CountVectorizer for Classification

In [107]:
#tri-gram CountVectorizer

trigram_vectorizer = CountVectorizer(ngram_range=(3,3)).fit_transform(utterances['lemmatized_utterance'])
X = np.array(trigram_vectorizer.todense())
print("Shape of X: ", X.shape)
y = utterances['intent']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)
print("Shape of X_train: ", X_train.shape)
print("Shape of X_test: ", X_test.shape)


Shape of X:  (8618, 20985)
Shape of X_train:  (6894, 20985)
Shape of X_test:  (1724, 20985)


In [108]:
ctvectorizer_trigram_MNB_model = MultinomialNB().fit(X_train, y_train)
y_pred = ctvectorizer_trigram_MNB_model.predict(X_test)

y_pred

array(['card_payment_fee_charged', 'transaction_charged_twice',
       'top_up_by_cash_or_cheque', ..., 'card_payment_fee_charged',
       'getting_spare_card', 'card_payment_fee_charged'], dtype='<U48')

In [109]:
print('Accuracy:', accuracy_score(y_test, y_pred))
print('F1 score:', f1_score(y_test, y_pred, average="macro"))

print (classification_report(y_test, y_pred))

Accuracy: 0.4547563805104408
F1 score: 0.5387573493757783
                                                  precision    recall  f1-score   support

                           Refund_not_showing_up       0.82      0.31      0.45        29
                                activate_my_card       0.57      0.46      0.51        28
                                       age_limit       1.00      0.39      0.56        18
                         apple_pay_or_google_pay       1.00      0.73      0.84        22
                                     atm_support       1.00      0.29      0.44        14
                                automatic_top_up       1.00      0.50      0.67        22
         balance_not_updated_after_bank_transfer       0.74      0.57      0.64        30
balance_not_updated_after_cheque_or_cash_deposit       0.90      0.55      0.68        33
                         beneficiary_not_allowed       0.67      0.44      0.53        27
                                 cancel_t

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


###Model building using MultiNomialNB (Naive Bayes) Model and tf-idf (term frequency - inverse document frequency) vectorizer for Classification


In [110]:

# Using TfidfVectorizer

tfidfVectorizer = TfidfVectorizer().fit_transform(utterances['lemmatized_utterance'])
X = np.array(tfidfVectorizer.todense())

y = utterances['intent']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)
print("Shape of X_train: ", X_train.shape)
print("Shape of X_test: ", X_test.shape)


Shape of X_train:  (6894, 1811)
Shape of X_test:  (1724, 1811)


In [111]:
tfidfvectorizer_MNB_model = MultinomialNB().fit(X_train, y_train)
y_pred = tfidfvectorizer_MNB_model.predict(X_test)

y_pred

array(['wrong_amount_of_cash_received', 'pending_top_up',
       'visa_or_mastercard', ..., 'cancel_transfer',
       'wrong_exchange_rate_for_cash_withdrawal',
       'declined_cash_withdrawal'], dtype='<U48')

In [112]:
print('Accuracy:', accuracy_score(y_test, y_pred))
print('F1 score:', f1_score(y_test, y_pred, average="macro"))

print (classification_report(y_test, y_pred))

Accuracy: 0.7848027842227379
F1 score: 0.738178697761854
                                                  precision    recall  f1-score   support

                           Refund_not_showing_up       0.89      0.86      0.88        29
                                activate_my_card       0.77      0.96      0.86        28
                                       age_limit       1.00      0.89      0.94        18
                         apple_pay_or_google_pay       0.96      1.00      0.98        22
                                     atm_support       1.00      0.14      0.25        14
                                automatic_top_up       1.00      0.95      0.98        22
         balance_not_updated_after_bank_transfer       0.49      0.80      0.61        30
balance_not_updated_after_cheque_or_cash_deposit       0.68      0.91      0.78        33
                         beneficiary_not_allowed       0.89      0.89      0.89        27
                                 cancel_tr

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [113]:
# tf-idf vectorizer and MNB model using Pipeline approach

X = utterances['lemmatized_utterance']
y = utterances['intent']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

print("Shape of X_train: ", X_train.shape)
print("Shape of X_test: ", X_test.shape)

# Creating a pipeline object
tfidf_vector_MNB_pipeline = Pipeline([
    ('vectorizer_tfidf', TfidfVectorizer()),        #using the ngram_range parameter 
     ('Multi NB', MultinomialNB())         
])
 
 #fit with features
tfidf_vector_MNB_pipeline.fit(X_train, y_train)



Shape of X_train:  (6894,)
Shape of X_test:  (1724,)


Pipeline(steps=[('vectorizer_tfidf', TfidfVectorizer()),
                ('Multi NB', MultinomialNB())])

In [114]:
#get the predictions for X_test using pipeline object

y_pred = tfidf_vector_MNB_pipeline.predict(X_test)
y_pred

array(['card_linking', 'Refund_not_showing_up', 'passcode_forgotten', ...,
       'automatic_top_up', 'disposable_card_limits', 'activate_my_card'],
      dtype='<U48')

In [115]:
print('Accuracy:', accuracy_score(y_test, y_pred))
print('F1 score:', f1_score(y_test, y_pred, average="macro"))

print (classification_report(y_test, y_pred))

Accuracy: 0.8033642691415314
F1 score: 0.7661596021734353
                                                  precision    recall  f1-score   support

                           Refund_not_showing_up       1.00      0.72      0.84        29
                                activate_my_card       0.78      1.00      0.88        28
                                       age_limit       1.00      0.89      0.94        18
                         apple_pay_or_google_pay       0.92      1.00      0.96        22
                                     atm_support       1.00      0.29      0.44        14
                                automatic_top_up       1.00      0.91      0.95        22
         balance_not_updated_after_bank_transfer       0.61      0.81      0.69        31
balance_not_updated_after_cheque_or_cash_deposit       0.73      0.97      0.83        33
                         beneficiary_not_allowed       0.90      0.70      0.79        27
                                 cancel_t

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Using KNN (K Nearest Neighbour) Model and CountVectorizer

##### Using ngram_range as unigram and bigram and metric as Euclidean distance.

In [116]:
# CountVectorizer and KNN model using Pipeline approach 

X = utterances['lemmatized_utterance']
y = utterances['intent']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

# Creating a pipeline object
count_vector_KNN1_pipeline = Pipeline([
    ('vectorizer_count', CountVectorizer(ngram_range=(1,2))),        #using the ngram_range parameter 
     ('KNN', KNeighborsClassifier(n_neighbors=5, weights='distance', metric='euclidean'))         
])
 
 #fit with features
count_vector_KNN1_pipeline.fit(X_train, y_train)

#get the predictions of X_test using pieline object
y_pred = count_vector_KNN1_pipeline.predict(X_test)

print('Accuracy:', accuracy_score(y_test, y_pred))
print('F1 score:', f1_score(y_test, y_pred, average="macro"))

print (classification_report(y_test, y_pred))

Accuracy: 0.66415313225058
F1 score: 0.6698008958767016
                                                  precision    recall  f1-score   support

                           Refund_not_showing_up       0.69      0.69      0.69        29
                                activate_my_card       0.70      0.82      0.75        28
                                       age_limit       0.46      0.89      0.60        18
                         apple_pay_or_google_pay       0.96      1.00      0.98        22
                                     atm_support       0.29      0.71      0.41        14
                                automatic_top_up       0.88      0.95      0.91        22
         balance_not_updated_after_bank_transfer       0.37      0.87      0.52        31
balance_not_updated_after_cheque_or_cash_deposit       0.86      0.55      0.67        33
                         beneficiary_not_allowed       0.76      0.48      0.59        27
                                 cancel_tra

##### Using ngram_range as unigram, bigram and trigram and metric as cosine distance.

In [117]:
# CountVectorizer and KNN model using Pipeline approach 

X = utterances['lemmatized_utterance']
y = utterances['intent']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

# Creating a pipeline object
count_vector_KNN2_pipeline = Pipeline([
    ('vectorizer_count', CountVectorizer(ngram_range=(1,3))),        #using the ngram_range parameter 
     ('KNN', KNeighborsClassifier(n_neighbors=5, weights='distance', metric='cosine'))         
])
 
 #fit with features
count_vector_KNN2_pipeline.fit(X_train, y_train)

#get the predictions of X_test using pieline object
y_pred = count_vector_KNN2_pipeline.predict(X_test)

print('Accuracy:', accuracy_score(y_test, y_pred))
print('F1 score:', f1_score(y_test, y_pred, average="macro"))

print (classification_report(y_test, y_pred))

Accuracy: 0.7587006960556845
F1 score: 0.7508097289568746
                                                  precision    recall  f1-score   support

                           Refund_not_showing_up       0.90      0.90      0.90        29
                                activate_my_card       0.79      0.68      0.73        28
                                       age_limit       0.94      0.89      0.91        18
                         apple_pay_or_google_pay       0.91      0.95      0.93        22
                                     atm_support       0.65      0.79      0.71        14
                                automatic_top_up       0.90      0.86      0.88        22
         balance_not_updated_after_bank_transfer       0.49      0.65      0.56        31
balance_not_updated_after_cheque_or_cash_deposit       0.84      0.79      0.81        33
                         beneficiary_not_allowed       0.95      0.70      0.81        27
                                 cancel_t

### Using KNN (K Nearest Neighbour) Model and TF-IDF Vectorizer

#####Using Euclidean distance metric.

In [118]:
# TF-IDF Vectorizer and KNN model using Pipeline approach 

X = utterances['lemmatized_utterance']
y = utterances['intent']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

# Creating a pipeline object
tfidf_vector_KNN1_pipeline = Pipeline([
    ('vectorizer_tfidf', TfidfVectorizer()),       
     ('KNN', KNeighborsClassifier(n_neighbors=10, weights='distance', metric='euclidean'))         
])
 
 #fit with features
tfidf_vector_KNN1_pipeline.fit(X_train, y_train)

#get the predictions of X_test using pieline object
y_pred = tfidf_vector_KNN1_pipeline.predict(X_test)

print('Accuracy:', accuracy_score(y_test, y_pred))
print('F1 score:', f1_score(y_test, y_pred, average="macro"))

print (classification_report(y_test, y_pred))

Accuracy: 0.7853828306264501
F1 score: 0.78157920791717
                                                  precision    recall  f1-score   support

                           Refund_not_showing_up       0.86      0.83      0.84        29
                                activate_my_card       1.00      0.93      0.96        28
                                       age_limit       1.00      1.00      1.00        18
                         apple_pay_or_google_pay       1.00      1.00      1.00        22
                                     atm_support       0.69      0.64      0.67        14
                                automatic_top_up       1.00      0.82      0.90        22
         balance_not_updated_after_bank_transfer       0.63      0.77      0.70        31
balance_not_updated_after_cheque_or_cash_deposit       0.89      0.94      0.91        33
                         beneficiary_not_allowed       0.78      0.67      0.72        27
                                 cancel_tra

#####Using Cosine distance metric.

In [119]:
# TF-IDF Vectorizer and KNN model using Pipeline approach 

X = utterances['lemmatized_utterance']
y = utterances['intent']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

# Creating a pipeline object
tfidf_vector_KNN2_pipeline = Pipeline([
    ('vectorizer_tfidf', TfidfVectorizer()),       
     ('KNN', KNeighborsClassifier(n_neighbors=10, weights='distance', metric='cosine'))         
])
 
 #fit with features
tfidf_vector_KNN2_pipeline.fit(X_train, y_train)

#get the predictions of X_test using pieline object
y_pred = tfidf_vector_KNN2_pipeline.predict(X_test)

print('Accuracy:', accuracy_score(y_test, y_pred))
print('F1 score:', f1_score(y_test, y_pred, average="macro"))

print (classification_report(y_test, y_pred))

Accuracy: 0.7801624129930395
F1 score: 0.7708850336695414
                                                  precision    recall  f1-score   support

                           Refund_not_showing_up       0.86      0.83      0.84        29
                                activate_my_card       0.96      0.96      0.96        28
                                       age_limit       0.95      1.00      0.97        18
                         apple_pay_or_google_pay       1.00      0.95      0.98        22
                                     atm_support       0.83      0.71      0.77        14
                                automatic_top_up       1.00      1.00      1.00        22
         balance_not_updated_after_bank_transfer       0.71      0.77      0.74        31
balance_not_updated_after_cheque_or_cash_deposit       0.90      0.79      0.84        33
                         beneficiary_not_allowed       0.95      0.70      0.81        27
                                 cancel_t

### Using SVM (Support Vector Machine) Model and TF-IDF Vectorizer

##### Using TF-IDF Vectorizer

In [120]:
# TF-IDF Vectorizer and SVM model 

tfidfVectorizer = TfidfVectorizer().fit_transform(utterances['lemmatized_utterance'])
X = np.array(tfidfVectorizer.todense())

y = utterances['intent']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

SVM_classifier_model1 = SVC(kernel='linear', gamma='auto')

#fit with features
SVM_classifier_model1.fit(X_train, y_train)

#get the predictions of X_test using pieline object
y_pred = SVM_classifier_model1.predict(X_test)

print('Accuracy:', accuracy_score(y_test, y_pred))
print('F1 score:', f1_score(y_test, y_pred, average="macro"))

print (classification_report(y_test, y_pred))


Accuracy: 0.845707656612529
F1 score: 0.8410183655731164
                                                  precision    recall  f1-score   support

                           Refund_not_showing_up       0.76      0.86      0.81        29
                                activate_my_card       0.96      0.93      0.95        28
                                       age_limit       1.00      1.00      1.00        18
                         apple_pay_or_google_pay       0.96      1.00      0.98        22
                                     atm_support       0.75      0.86      0.80        14
                                automatic_top_up       1.00      0.95      0.98        22
         balance_not_updated_after_bank_transfer       0.72      0.68      0.70        31
balance_not_updated_after_cheque_or_cash_deposit       0.89      0.94      0.91        33
                         beneficiary_not_allowed       0.89      0.93      0.91        27
                                 cancel_tr

##### Using Count Vectorizer

In [121]:
# Count Vectorizer with unigram and bigram range and SVC model

count_vectorizer = CountVectorizer(ngram_range=(1,2)).fit_transform(utterances['lemmatized_utterance'])
X = np.array(count_vectorizer.todense())
y = utterances['intent']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

SVM_classifier_mode2 = SVC(kernel='linear', gamma='auto')

#fit with features
SVM_classifier_mode2.fit(X_train, y_train)

#get the predictions of X_test using pieline object
y_pred = SVM_classifier_mode2.predict(X_test)

print('Accuracy:', accuracy_score(y_test, y_pred))
print('F1 score:', f1_score(y_test, y_pred, average="macro"))

print (classification_report(y_test, y_pred))


Accuracy: 0.8230858468677494
F1 score: 0.8185962862533057
                                                  precision    recall  f1-score   support

                           Refund_not_showing_up       0.93      0.90      0.91        29
                                activate_my_card       1.00      0.86      0.92        28
                                       age_limit       0.94      0.94      0.94        18
                         apple_pay_or_google_pay       1.00      1.00      1.00        22
                                     atm_support       0.58      0.79      0.67        14
                                automatic_top_up       1.00      0.95      0.98        22
         balance_not_updated_after_bank_transfer       0.65      0.73      0.69        30
balance_not_updated_after_cheque_or_cash_deposit       1.00      0.97      0.98        33
                         beneficiary_not_allowed       0.82      0.85      0.84        27
                                 cancel_t

### Using Random Forest (Reinforcement Learning) Model

### Using Gensim library for text classification

### Performing NER (Named Entity Recognition